In [1]:
from bert_multitask_learning import train_bert_multitask, train_eval_input_fn, BertMultiTask, params

In [2]:
cd ../

/data3/yjp/bert-multitask-learning


## Train Models
If you don't want to control every thing, you can just call `train_bert_multitask` function. 

In [3]:
train_bert_multitask(problem='weibo_ner&weibo_cws', num_gpus=1, num_epochs=3)

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:3
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:3
INFO:tensorflow:Configured nccl all-reduce.
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Using config: {'_m

/data3/yjp/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:batch_all_reduce invoked for batches size = 201 with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into models/weibo_cws_weibo_ner_ckpt/model.ckpt.
INFO:tensorflow:Initialize strategy
INFO:tensorflow:loss = 3.9036303, step = 0
INFO:tensorflow:learning_rate = 0.0, total_training_steps = 253, weibo_cws_loss = 1.4725939, weibo_ner_loss = 2.4310365
INFO:tensorflow:global_step/sec: 1.92174
INFO:tensorflow:loss = 0.09233739, step = 100 (52.044 sec)
INFO:tensorflow:learning_rate = 1.2094861e-05, total_training_steps = 253, weibo_cws_loss = 0.06148266, weibo_ner_loss = 0.030854732 (52.040 sec)
INFO:tensorflow:global_step/sec: 2.19173
INFO:tensorflow:loss = 0.03317994, step = 200 (45.609 

If you want to take more control of the training process, you can use lower level api

In [4]:
import tensorflow as tf
from tensorflow.estimator import Estimator
from bert_multitask_learning.ckpt_restore_hook import RestoreCheckpointHook

problem = 'weibo_ner&weibo_cws'
num_gpus = 1
bert_multitask_params = params.DynamicBatchSizeParams()

# assign problem to params
bert_multitask_params.train_epoch = 5
bert_multitask_params.assign_problem(problem, gpu=num_gpus)

In [5]:
# get model fn and create mirror strategy for distributed training
model = BertMultiTask(params=bert_multitask_params)
model_fn = model.get_model_fn()

dist_trategy = tf.contrib.distribute.MirroredStrategy(
    num_gpus=int(num_gpus),
    cross_tower_ops=tf.contrib.distribute.AllReduceCrossDeviceOps(
        'nccl', num_packs=int(num_gpus)))

run_config = tf.estimator.RunConfig(
    train_distribute=dist_trategy,
    eval_distribute=dist_trategy,
    log_step_count_steps=bert_multitask_params.log_every_n_steps)

# create estimator
estimator = Estimator(
    model_fn,
    model_dir=bert_multitask_params.ckpt_dir,
    params=bert_multitask_params,
    config=run_config)

# pretrained bert restore hook
train_hook = RestoreCheckpointHook(bert_multitask_params)

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:3
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:3
INFO:tensorflow:Configured nccl all-reduce.
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Using config: {'_m

In [6]:
# train
def train_input_fn(): return train_eval_input_fn(bert_multitask_params)
estimator.train(
    train_input_fn, max_steps=bert_multitask_params.train_steps, hooks=[train_hook])


INFO:tensorflow:{'input_ids': tf.int32, 'input_mask': tf.int32, 'segment_ids': tf.int32, 'weibo_ner_loss_multiplier': tf.int32, 'weibo_ner_label_ids': tf.int32, 'weibo_cws_loss_multiplier': tf.int32, 'weibo_cws_label_ids': tf.int32}
INFO:tensorflow:{'input_ids': [None], 'input_mask': [None], 'segment_ids': [None], 'weibo_ner_loss_multiplier': [], 'weibo_ner_label_ids': [None], 'weibo_cws_loss_multiplier': [], 'weibo_cws_label_ids': [None]}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:batch_all_reduce invoked for batches size = 201 with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/weibo_cws_weibo_ner_ckpt/model.ckpt-253
Instructions for updating:
Use standard file utilities to g

## Evaluate and Predict

For NER and CWS, we need different evaluation logic.

In [7]:
from bert_multitask_learning import eval_bert_multitask, predict_bert_multitask

In [8]:
eval_bert_multitask(problem='weibo_cws', model_dir='models/weibo_cws_weibo_ner_ckpt/', eval_scheme='acc')

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:3
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:3
INFO:tensorflow:Configured nccl all-reduce.
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Using config: {'_m

Processing Inputs: 100%|██████████| 270/270 [00:00<00:00, 278.92it/s]


{'weibo_cws_Accuracy': 0.9494476297021396,
 'weibo_cws_Accuracy Per Sequence': 0.35555555555555557}

In [9]:
eval_bert_multitask(problem='weibo_ner', model_dir='models/weibo_cws_weibo_ner_ckpt/', eval_scheme='ner')

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:3
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:3
INFO:tensorflow:Configured nccl all-reduce.
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Using config: {'_m

Processing Inputs: 100%|██████████| 270/270 [00:00<00:00, 285.93it/s]


Acc Score: 0.981751
Precision Score: 0.678899
Recall Score: 0.691589
F1 Score: 0.685185


{'Acc': 0.981750804083345,
 'Precision': 0.6788990825688074,
 'Recall': 0.6915887850467289,
 'F1': 0.6851851851851852}

In [10]:
# predict
import numpy as np
from bert_multitask_learning.utils import get_or_make_label_encoder
predict_params = params.DynamicBatchSizeParams()
# get prediction generator
pred_prob = predict_bert_multitask(inputs=['中国和美国在打贸易战'], problem='weibo_cws&weibo_ner', params=predict_params)
# get label encoder
ner_label_encoder = get_or_make_label_encoder(params=predict_params, problem='weibo_ner', mode='predict')
cws_label_encoder = get_or_make_label_encoder(params=predict_params, problem='weibo_cws', mode='predict')

for prob in pred_prob:
    ner_pred = np.argmax(prob['weibo_ner'], axis = -1)
    print(ner_label_encoder.inverse_transform(ner_pred.tolist()))

INFO:tensorflow:Checkpoint dir: models/weibo_cws_weibo_ner_ckpt
INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:3
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:1
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:2
INFO:tensorflow:Device is available but not used by distribute strategy: /device:GPU:3
INFO:tensorflow:Configured nccl all-reduce.
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not 

Processing Inputs: 100%|██████████| 1/1 [00:00<00:00, 563.98it/s]

['[PAD]' 'B-GPE' 'I-GPE' 'O' 'B-GPE' 'I-GPE' 'O' 'O' 'O' 'O' 'O' '[PAD]']


## Export Model for Serving

You can export the trained model for [serving](https://github.com/JayYip/bert-as-service).

In [11]:
from bert_multitask_learning import export_model

In [12]:
export_model(bert_multitask_params)

INFO:tensorflow:build graph...
INFO:tensorflow:load parameters from checkpoint...
INFO:tensorflow:freeze...
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 201 variables.
INFO:tensorflow:Converted 201 variables to const ops.
INFO:tensorflow:write graph to: models/weibo_cws_weibo_ner_ckpt/export_model
